In [1]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI='sqlite:///mlflow.db'
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [2]:
client.create_experiment(name='my-cool-experiment')

MlflowException: Experiment(name=my-cool-experiment) already exists. Error: (raised as a result of Query-invoked autoflush; consider using a session.no_autoflush block if this flush is occurring prematurely)
(sqlite3.IntegrityError) UNIQUE constraint failed: experiments.name
[SQL: INSERT INTO experiments (name, artifact_location, lifecycle_stage, creation_time, last_update_time) VALUES (?, ?, ?, ?, ?)]
[parameters: ('my-cool-experiment', None, 'active', 1743305298178, 1743305298178)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [71]:
from mlflow.entities import ViewType
runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 5.3",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=10,
    order_by=['metrics.rmse ASC']
)

In [72]:
for run in runs:
    print(f'run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}')

run id: fea3153267d44bc58128f08d8ff4eb36, rmse: 5.1250
run id: 70ac6f7b08ef42b3b4e7115810836c49, rmse: 5.1250
run id: 5c8593ac4f5245c58a4c1284c799ef99, rmse: 5.1250
run id: 7af79c48326e4d4d8a7dfab73c02992c, rmse: 5.1250
run id: 594fb116fd0f4636b187731d227b3b9e, rmse: 5.1543


In [5]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)


In [84]:
run_id='fea3153267d44bc58128f08d8ff4eb36'

model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '5' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1743309795666, current_stage='None', description=None, last_updated_timestamp=1743309795666, name='nyc-taxi-regressor', run_id='fea3153267d44bc58128f08d8ff4eb36', run_link=None, source='file:c:/Users/rozak/Downloads/projects/My-MLOps-Zoomcamp/mlruns/1/fea3153267d44bc58128f08d8ff4eb36/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=5>

In [ ]:
model_name='nyc-taxi-regressor'
client.get_latest_versions(name=model_name)

C:\Users\rozak\AppData\Local\Temp\ipykernel_15092\2394194884.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.get_latest_versions(name=model_name)


[<ModelVersion: aliases=[], creation_timestamp=1743309569112, current_stage='None', description=None, last_updated_timestamp=1743309569112, name='nyc-taxi-regressor', run_id='fea3153267d44bc58128f08d8ff4eb36', run_link=None, source='file:c:/Users/rozak/Downloads/projects/My-MLOps-Zoomcamp/mlruns/1/fea3153267d44bc58128f08d8ff4eb36/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>,
 <ModelVersion: aliases=['champion'], creation_timestamp=1743305721499, current_stage='Staging', description='the model version 3 was transitioned to champion on 2025-03-30', last_updated_timestamp=1743306981828, name='nyc-taxi-regressor', run_id='70ac6f7b08ef42b3b4e7115810836c49', run_link=None, source='file:c:/Users/rozak/Downloads/projects/My-MLOps-Zoomcamp/mlruns/1/70ac6f7b08ef42b3b4e7115810836c49/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>]

In [75]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 4, stage: None
version: 3, stage: Staging


C:\Users\rozak\AppData\Local\Temp\ipykernel_15092\669935608.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [76]:
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage='staging',
    archive_existing_versions=False
)

C:\Users\rozak\AppData\Local\Temp\ipykernel_15092\845015496.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1743309569112, current_stage='Staging', description=None, last_updated_timestamp=1743309583069, name='nyc-taxi-regressor', run_id='fea3153267d44bc58128f08d8ff4eb36', run_link=None, source='file:c:/Users/rozak/Downloads/projects/My-MLOps-Zoomcamp/mlruns/1/fea3153267d44bc58128f08d8ff4eb36/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [ ]:
client.transition_model_version_stage(
    name=model_name,
    version=4,
    archive_existing_versions=False
)

In [110]:
version=6
alias='champion'
stage='production'

client.transition_model_version_stage(
    name=model_name,
    version=version,
    stage=stage,
    archive_existing_versions=False
)

client.set_registered_model_alias(
    name=model_name,
    version=version,
    alias=alias
)

C:\Users\rozak\AppData\Local\Temp\ipykernel_15092\2063000445.py:5: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


In [111]:
from datetime import datetime

date=datetime.today().date()
client.update_model_version(
    name=model_name,
    version=version,
    description=f"the model version {version} was transitioned to {alias} and {stage} on {date}"
)

<ModelVersion: aliases=['champion'], creation_timestamp=1743310525912, current_stage='Production', description='the model version 6 was transitioned to champion and production on 2025-03-30', last_updated_timestamp=1743310567434, name='nyc-taxi-regressor', run_id=None, run_link=None, source='file:c:/Users/rozak/Downloads/projects/My-MLOps-Zoomcamp/mlruns/1/fea3153267d44bc58128f08d8ff4eb36/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=6>

In [36]:
import pickle
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [38]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [39]:
def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


In [92]:
import numpy as np
def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": np.sqrt(mean_squared_error(y_test, y_pred))}

In [55]:
df = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-03.parquet')

In [88]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'c:\\Users\\rozak\\Downloads\\projects\\My-MLOps-Zoomcamp\\preprocessor'

In [107]:
model = mlflow.pyfunc.load_model("file:/Users/rozak/Downloads/projects/My-MLOps-Zoomcamp/mlruns/1/fea3153267d44bc58128f08d8ff4eb36/artifacts/models_mlflow")


In [108]:
run_id = "fea3153267d44bc58128f08d8ff4eb36"
correct_model_uri = f"file:c:/Users/rozak/Downloads/projects/My-MLOps-Zoomcamp/mlruns/1/{run_id}/artifacts/models_mlflow"

mlflow.register_model(model_uri=correct_model_uri, name="nyc-taxi-regressor")


Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '6' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1743310525912, current_stage='None', description=None, last_updated_timestamp=1743310525912, name='nyc-taxi-regressor', run_id=None, run_link=None, source='file:c:/Users/rozak/Downloads/projects/My-MLOps-Zoomcamp/mlruns/1/fea3153267d44bc58128f08d8ff4eb36/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=6>

In [89]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [90]:
X_test = preprocess(df, dv)
target = "duration"
y_test = df[target].values

In [104]:
import mlflow

client = mlflow.tracking.MlflowClient()
models = client.search_model_versions("name='nyc-taxi-regressor'")

for model in models:
    print(f"Version: {model.version}, Stage: {model.current_stage}, Source: {model.source}")


Version: 5, Stage: Production, Source: file:c:/Users/rozak/Downloads/projects/My-MLOps-Zoomcamp/mlruns/1/fea3153267d44bc58128f08d8ff4eb36/artifacts/model
Version: 4, Stage: Staging, Source: file:c:/Users/rozak/Downloads/projects/My-MLOps-Zoomcamp/mlruns/1/fea3153267d44bc58128f08d8ff4eb36/artifacts/model
Version: 3, Stage: Staging, Source: file:c:/Users/rozak/Downloads/projects/My-MLOps-Zoomcamp/mlruns/1/70ac6f7b08ef42b3b4e7115810836c49/artifacts/model
Version: 2, Stage: None, Source: file:c:/Users/rozak/Downloads/projects/My-MLOps-Zoomcamp/mlruns/1/70ac6f7b08ef42b3b4e7115810836c49/artifacts/models_mlflow
Version: 1, Stage: None, Source: file:c:/Users/rozak/Downloads/projects/My-MLOps-Zoomcamp/mlruns/1/5c8593ac4f5245c58a4c1284c799ef99/artifacts/models_mlflow


In [112]:
%time test_model(name="nyc-taxi-regressor", stage="Production", X_test=X_test, y_test=y_test)

CPU times: total: 4min 32s
Wall time: 33.9 s


{'rmse': np.float64(5.458531963614643)}

In [113]:
%time test_model(name="nyc-taxi-regressor", stage="Staging", X_test=X_test, y_test=y_test)

OSError: No such file or directory: 'C:\Users\rozak\Downloads\projects\My-MLOps-Zoomcamp\mlruns\1\fea3153267d44bc58128f08d8ff4eb36\artifacts\model\.'

In [100]:
from mlflow import MlflowClient

client = MlflowClient()
models = client.search_registered_models()

for model in models:
    print(model.name)


nyc-taxi-regressor


In [101]:
versions = client.get_latest_versions(name="nyc-taxi-regressor", stages=["Production"])
print(versions)


[<ModelVersion: aliases=['champion'], creation_timestamp=1743309795666, current_stage='Production', description='the model version 5 was transitioned to champion and production on 2025-03-30', last_updated_timestamp=1743310200890, name='nyc-taxi-regressor', run_id='fea3153267d44bc58128f08d8ff4eb36', run_link=None, source='file:c:/Users/rozak/Downloads/projects/My-MLOps-Zoomcamp/mlruns/1/fea3153267d44bc58128f08d8ff4eb36/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=5>]


C:\Users\rozak\AppData\Local\Temp\ipykernel_15092\2588058796.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  versions = client.get_latest_versions(name="nyc-taxi-regressor", stages=["Production"])


In [102]:
versions = client.get_latest_versions(name="nyc-taxi-regressor")
for v in versions:
    print(f"Version: {v.version}, Stage: {v.current_stage}")


Version: 2, Stage: None
Version: 4, Stage: Staging
Version: 5, Stage: Production


C:\Users\rozak\AppData\Local\Temp\ipykernel_15092\1000357056.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  versions = client.get_latest_versions(name="nyc-taxi-regressor")
